In [1]:
import fileinput
import string
from collections import defaultdict, Counter
from collections.abc import Callable
from itertools import filterfalse
import random

https://www.nytimes.com/games/wordle/index.html

In [2]:
def inc_exc(words: list[str], 
            letter_groups: dict[str, set[str]], 
            include: str, 
            exclude: str) -> list[str]:
    """Filters words based on the characters in include and exclude.
    
    If include and exclude are empty, inc_exc returns the same 
    list of words, albeit a copy of the original list with duplicate words removed.
    
    Args:
        words: A list of words.
        letter_groups: A mapping between a character in a word and 
            a subset from words that contain that character.
        include: A string of characters that must be contained in all the output words.
        exclude: A string of characters that must _not_ be contained in any output words.
        
    Returns:
        A list of filtered words where each word contains all the characters in include, but 
        does not have the characters listed in exclude.
    """
    include, exclude = set(include), set(exclude)
    possible = set(words)
    
    for ch in include:
        possible &= letter_groups[ch]

    print(f'Words that include {include}: {len(possible):,}')
    for ch in exclude:
        possible -= letter_groups[ch]

    print(f'Words that include {include}, but exclude {exclude}: {len(possible):,}')
    
    return list(possible)

def group_by_letter(words: list[str], char_set: str=string.ascii_lowercase) -> dict[str, set[str]]:
    """Groups words into sets based on what character is contained in the word.
    
    Args:
        words: A list of words.
        char_set: A string containing all the characters used to compose words.
        
    Returns:
        A mapping between each character in char_set and 
        a subset of words that contain that character.
    """
    letters = defaultdict(set)
    for ch in char_set:
        for word in words:
            if ch in word:
                letters[ch].add(word)
    return letters

class Possible(object):
    def __init__(self, iterable: list[str]):
        print(f'Sorting values')
        iterable.sort()
        self.values = iterable
    
    def distinct_letters_only(self):
        self.filter(lambda word: len(set(word)) == len(word))
        
    def filter(self, condition: Callable[[str], bool]):
        self.values = list(filter(condition, self.values))
        
    def __repr__(self):
        return f'Possible({self.values})'
    
    def __len__(self):
        return len(self.values)
        

In [3]:
words = [word.strip() for word in fileinput.input('words_alpha.txt')]
print(f'All words: {len(words):,}')

five_letter_words = [word for word in words if len(word) == 5]
five_letters_grouped = group_by_letter(five_letter_words)
five = lambda inc, exc: inc_exc(five_letter_words, five_letters_grouped, inc, exc)

print(f'Five-letter words: {len(five_letter_words):,}')

All words: 370,105
Five-letter words: 15,920


## Spelling Bee

In [4]:
grouped = group_by_letter(words)
all_words = lambda inc, exc: inc_exc(words, grouped, inc, exc)

In [21]:
excluded_letters = ''.join(sorted(set(string.ascii_lowercase) - set('belatedly')))
excluded_letters

'cfghijkmnopqrsuvwxz'

In [20]:
a = all_words('y', excluded_letters)
a = list(filter(lambda word: 4 <= len(word) <= 100, a))

a.sort()
a.sort(key=len, reverse=True)

print(len(a))
a[:20]

Words that include {'y'}: 66,545
Words that include {'y'}, but exclude {'n', 'x', 'q', 'h', 'i', 'r', 'm', 's', 'g', 'p', 'k', 'u', 'o', 'w', 'f', 'c', 'z', 'v', 'j'}: 188
151


['telltalely',
 'belatedly',
 'debatably',
 'delayable',
 'eyeballed',
 'eyeletted',
 'allylate',
 'attababy',
 'beatably',
 'elatedly',
 'eyeleted',
 'addedly',
 'allayed',
 'alleyed',
 'atalaya',
 'beadeye',
 'belayed',
 'blately',
 'datably',
 'datedly']

## Guesses

### Method 1: Uniformally random word

In [6]:
distinct_letters = list(filter(lambda word: len(set(word)) == len(word), five_letter_words))

In [7]:
random.choice(distinct_letters)

'lieus'

### Method 2: Pick words with letters that occur the most

The end goal is to eliminate as many words as possible.  For the first guess, if we pick a word that has letters that occur the most, we can see if any of those letters can be discarded.  Because we used the most frequently used letters, we theoretically eliminate more words per letter using these letters.

In [8]:
hist = Counter()
for word in five_letter_words:
    for ch in word:
        hist[ch] += 1

In [9]:
hist.most_common(10)

[('a', 8393),
 ('e', 7802),
 ('s', 6537),
 ('o', 5219),
 ('r', 5144),
 ('i', 5067),
 ('l', 4247),
 ('t', 4189),
 ('n', 4044),
 ('u', 3361)]

In [10]:
possible = five('arilt', '')
possible

Words that include {'r', 'l', 'i', 'a', 't'}: 3
Words that include {'r', 'l', 'i', 'a', 't'}, but exclude set(): 3


['trial', 'trail', 'litra']

## Filter words

In [11]:
print(f'Words with five letters: {len(five_letter_words):,}')

Words with five letters: 15,920


In [14]:
words = Possible(five('tal', 'rime'))

words.filter(lambda word: word[0] != 't' and word[2] != 'a' and word[4] == 'l')
words.filter(lambda word: word[2] != 't' and word[3] != 'a')
# words.distinct_letters_only()

print(len(words))
words

Words that include {'t', 'a', 'l'}: 285
Words that include {'t', 'a', 'l'}, but exclude {'e', 'r', 'i', 'm'}: 128
Sorting values
1


Possible(['atoll'])

## Save results

In [15]:
import sqlite3
from collections import namedtuple
import itertools

Result = namedtuple('Result', ['wordle', 'date', 'words', 'regular', 'high_contrast'])
def println(*objects, **kw):
    print(*objects, end='\n\n', **kw)
    
def print_results(results, show_rowid=False):
    def header(head, rowid):
        msg = f'{head}'
        if show_rowid:
            msg += f' rowid: {rowid}'
            
        return msg
        
    for rowid, *result in results:
        result = Result(*result)

        hi = result.high_contrast.splitlines()
        words = result.words.splitlines()
        
        print(header(hi[0], rowid))
        for line, word in itertools.zip_longest(itertools.islice(hi, 2, None), words, fillvalue=''):
            print(f'{line} {word}')
        print()
    
con = sqlite3.connect('wordle.db')
cur = con.cursor()

In [19]:
most_recent = cur.execute('''SELECT ROWID, * FROM results ORDER BY ROWID DESC''').fetchall()[:5]
print_results(most_recent)

Wordle 345 3/6*
🟦⬜🟦⬜🟧 trail
⬜⬜🟦🟦🟧 metal
🟧🟧🟧🟧🟧 atoll

Wordle 344 4/6*
⬜⬜⬜🟦⬜ clear
⬜⬜⬜⬜🟦 vista
🟧🟧⬜⬜🟦 banjo
🟧🟧🟧🟧🟧 bayou

Wordle 342 3/6*
⬜🟧🟦⬜🟦 liter
⬜🟧🟦🟦⬜ birth
🟧🟧🟧🟧🟧 tiara

Wordle 341 5/6*
⬜⬜⬜⬜⬜ bound
🟧⬜⬜⬜🟦 alive
🟧🟧⬜🟧⬜ askew
🟧🟧⬜🟧🟦 ashes
🟧🟧🟧🟧🟧 asset

Wordle 340 6/6*
⬜🟦⬜⬜⬜ jubes
⬜⬜🟧⬜⬜ adult
⬜🟧🟧⬜⬜ mourn
🟦🟧🟧⬜🟧 cough
⬜🟧🟧🟧🟧 pouch
🟧🟧🟧🟧🟧 vouch



### Create table

Only need to do once.  Uncomment to execute.

In [17]:
# cur.execute('''
# CREATE TABLE results 
# (wordle, date, words, regular, high_contrast)
# ''')
# con.commit()

### Save result

To keep it simple, everything is stored as TEXT.

In [18]:
cur.execute('''
INSERT INTO results VALUES (
'345', '2022-05-30', 
'trail
metal
atoll', 
'Wordle 345 3/6*

🟨⬜🟨⬜🟩
⬜⬜🟨🟨🟩
🟩🟩🟩🟩🟩', 
'Wordle 345 3/6*

🟦⬜🟦⬜🟧
⬜⬜🟦🟦🟧
🟧🟧🟧🟧🟧'
)''')

con.commit()

### SQL utilities

In [22]:
# cur.execute('''SELECT * FROM results WHERE wordle='325' ''').fetchall()

In [44]:
# cur.execute('''DELETE FROM results WHERE wordle='325' ''')
# con.commit()

In [20]:
# close connection to database
con.close()